In [1]:
import geemap 

Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [2]:
import pandas as pd
import os
from pathlib import Path

#lire le fichier de points 
file = os.path.expanduser('~') + '/test_pts.txt'

filename = Path(file).stem

pts = pd.read_csv(file)
#pts

In [3]:
import ee

ee.Initialize()

ee_pts = [ee.Geometry.Point(pts.loc[i]['lng'], pts.loc[i]['lat']) for i in range(len(pts))]

In [4]:
buffers = [ee_pt.buffer(2000) for ee_pt in ee_pts]

#_ = [Map.addLayer(buffers[i], {}, str(i)) for i in range(len(buffers))]

In [5]:
from sepal_ui import widgetFactory as wf 

output = wf.OutputWidget('toto')
output

Alert(children=['toto'], class_='mt-5', text=True, type='info')

In [6]:
from utils import parameters as pm

bands = 'Red, Green, Blue'
name_bands = '_'.join(bands.split(', '))
name_bands

'Red_Green_Blue'

In [7]:
import time
from sepal_ui.scripts import gee
from utils import gdrive
import matplotlib.pyplot as plt
import tifffile
from sepal_ui.scripts import utils as su
from matplotlib.backends.backend_pdf import PdfPages
import rasterio as rio
import numpy as np
from utils import utils
import os


start_year = 2005
end_year = 2020
    
viz_bands = pm.getAvailableBands()[bands][0]
dataset_source = pm.getSources()['landsat 7']

drive_handler = gdrive.gdrive()


#export the data
pdf_file = pm.getResultDir() + '{}_{}.pdf'.format(filename, name_bands)
with PdfPages(pdf_file) as pdf:
    
    #each point will be a separate page 
    for index, row in pts.iterrows():
        
        #export the images 
        for year in range(start_year, end_year):
                
            start = str(year) + '-01-01';
            end = str(year) + '-12-31';
                
            dataset = ee.ImageCollection(dataset_source)
            dataset = dataset.filterDate(start, end).median()
            clip = dataset.clip(buffers[index]).select(viz_bands)
            
            description = '{}_pt_{}_{}_{}'.format(filename, index, name_bands, year)
                
            #export the image : 
            task_config = {
                'image':clip,
                'description':description,
                'scale': 30,
                'region':buffers[index],
                'maxPixels': 1e12
            }
        
            task = ee.batch.Export.image.toDrive(**task_config)
            task.start()
                
            
        #check download evolution
        task_names = ['{}_pt_{}_{}_{}'.format(filename, index, name_bands, year) for year in range(start_year, end_year)]
        state = utils.custom_wait_for_completion(task_names, output)
        su.displayIO(output, 'Download point {} finished'.format(index), 'success')
        
        #retreive to sepal
        for year in range(start_year, end_year):
            description = '{}_pt_{}_{}_{}'.format(filename, index, name_bands, year)
            files = drive_handler.get_files(description)
            drive_handler.download_files(files, pm.getTmpDir())

        #display the images in a fig and export it
        page_title = "Pt_{} (lat:{:.5f}, lng:{:.5f})".format(index, row['lat'], row['lng'])
        su.displayIO(output, 'Creating page for pt {}'.format(index))
        fig, axes = plt.subplots(5, 3, figsize=(8.27,11.69))
        fig.suptitle(page_title, fontsize=16)
        #plt.tight_layout()
        for year in range(start_year, end_year):
            description = '{}_pt_{}_{}_{}'.format(filename, index, name_bands, year)
            file = pm.getTmpDir() + description + '.tif'
            
            with rio.open(file) as f:
                data = f.read([1, 2, 3], masked=True)
                
            data = data/3000
            data = data.clip(0, 1)
            data = np.transpose(data,[1,2,0])
            
            i = year - start_year
            ax = axes[pm.getPositionPdf(i)[0], pm.getPositionPdf(i)[1]]
            ax.imshow(data)
            ax.set_title(str(year))
            ax.axis('off')
            
            #delete tmp file 
            os.remove(file)
        
        pdf.savefig(fig)
        plt.close()
        
        if index == 2:
            break